In [55]:
import xlrd
from pathlib import Path
import pandas as pd
import numpy as np
from numba import decorators
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset


In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [57]:
# read in the data file
# Give the location of the file 

df = pd.read_excel(r'data/data.xlsx', sheet_name='reduced totals')
# print(df)

In [66]:
## LOADING IN DATASETS

dataset = Path.cwd().joinpath("SongEmotionDataset")
datasheet = Path.cwd().joinpath("data") # for csua

#emotion labels
label_loc = datasheet.joinpath("data.xlsx")
wb = xlrd.open_workbook(label_loc) 
sheet = wb.sheet_by_index(2)

#emotion arr
emotions = ["amazement", "calmness", "power", "joyful activation", "sadness"]
# emotions = ["amazement", "solemnity", "tenderness", "nostalgia", "calmness", "power", "joyful activation", "tension", "sadness"]


train_song = []
test_song = []
train_emotion = []
test_emotion = []


row_totals = torch.zeros(len(emotions))
for i in range(1, 401):
    row_totals += 


for i in range(1, 401):
    count_total = sheet.cell_value(i, 7)
    
    emotions_counter = [0 for e in emotions]
    if i % 5 == 0:
        test_song.append(dataset.joinpath("{}.mp3".format(i)))
        emotion_arr = []
        for j in range(5):
            emotion_arr.append(sheet.cell_value(i, 2 + j))
        test_emotion.append(torch.tensor(emotion_arr, device=device).float())
        
    emotions_counter = [0 for e in emotions]
    else:
        train_song.append(dataset.joinpath("{}.mp3".format(i)))
        emotion_arr = []
        for j in range(5):
            emotion_arr.append(sheet.cell_value(i, 2 + j))
        train_emotion.append(torch.tensor(emotion_arr, device=device))

print(len(train_song), len(test_song))
print(len(train_emotion), len(test_emotion))

320 80
320 80


In [67]:
# train_emotion

In [68]:
class SongEmotionDataset(Dataset):
    """
    Song Emotion Dataset. Uses librosa to process mp3 files.
    Takes first 20 seconds, and samples every 10 to get processed audio tensor.
    """

    def __init__(self, mp3, labels, transform=None):
        """
        Args:
            mp3: list of paths to mp3 files
            labels: list of labels
        """
        self.labels = labels
        self.mp3 = mp3
        self.cache = {}
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if index not in self.cache.keys():
#             print("index of " + str(index) + " was cached!")
            data, rate = librosa.load(self.mp3[index], sr=16000, duration=10)
            mfccs = librosa.feature.mfcc(y=data, sr=rate, n_mfcc=40)
            assert rate == 16000
            sample_tensor = torch.tensor(mfccs, device=device).float()
            downsampled_tensor = sample_tensor[::10]
    #         print(mfccs.shape, data.shape)

            self.cache[index] = (downsampled_tensor, F.softmax(self.labels[index]))
#         else:
#             print("index was cached! index of " + str(index))
        return self.cache[index]

In [69]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (2,2))
        self.conv2 = nn.Conv2d(32, 64, (2,2))
        self.pool1 = nn.MaxPool2d((2,2))
        self.drop1 = nn.Dropout(p=0.25)
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(9920, 128)
        self.drop2 = nn.Dropout(p=0.5)
        self.dense2 = nn.Linear(128, 5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.flat1(x)
        x = self.dense1(x)
        x = F.relu(x)
        
        x = self.drop2(x)
        x = self.dense2(x)
        x = F.relu(x)
        
        return F.log_softmax(x, dim = 1)

model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.25, inplace=False)
  (flat1): Flatten()
  (dense1): Linear(in_features=9920, out_features=128, bias=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=128, out_features=9, bias=True)
)


In [70]:
train_set = SongEmotionDataset(train_song, train_emotion)
test_set = SongEmotionDataset(test_song, test_emotion)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

Train set size: 320
Test set size: 80


In [71]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data.unsqueeze_(1)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
#         output = output.view(-1, len(emotions))
#         print(output.shape, target.shape)
#         print(output, target)
        loss = F.kl_div(output, target)
#         loss = F.cross_entropy(output, target)
#         loss = nn.CrossEntropyLoss(output, target)
#         loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#         scheduler.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [72]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data. unsqueeze_(1)
        output = model(data)
        print(output)
        print(target)
        print("\n")
#         output = output.permute(1, 0, 2)
        pred = output.max(1)[1] # get the index of the max log-probability 
        correct += pred.eq(target.max(1)[1]).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [73]:
import warnings

log_interval = 5
warnings.filterwarnings("ignore")
for epoch in range(1, 100):
    print("training epoch " + str(epoch))
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
#     scheduler.step()
    train(model, epoch)
    scheduler.step()
    test(model, epoch)

training epoch 1
Train Epoch: 1 [0/320 (0%)]	Loss: 1.416775
Train Epoch: 1 [40/320 (12%)]	Loss: 0.212770
Train Epoch: 1 [80/320 (25%)]	Loss: 0.179154
Train Epoch: 1 [120/320 (38%)]	Loss: 0.207526
Train Epoch: 1 [160/320 (50%)]	Loss: 0.177037
Train Epoch: 1 [200/320 (62%)]	Loss: 0.174979
Train Epoch: 1 [240/320 (75%)]	Loss: 0.206796
Train Epoch: 1 [280/320 (88%)]	Loss: 0.163953
tensor([[-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2

tensor([[-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167],
        [-2.3167, -2.3167, -2.3167, -2.1630, -1.9139, -2.3167, -1.9247, -2.3167,
         -2.3167]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[8.9807e-05, 7.2771e-01, 6.6359e-04, 1.8038e-03, 2.6771e-01, 8.9807e-05,
         3.3038e-05, 8.9807e-05, 1.8038e-03],
        [1.6299e-03, 2.41

tensor([[-2.5170, -2.5170, -2.5170, -2.1990, -1.5619, -2.5170, -1.6340, -2.5170,
         -2.5170],
        [-2.5170, -2.5170, -2.5170, -2.1990, -1.5619, -2.5170, -1.6340, -2.5170,
         -2.5170],
        [-2.5170, -2.5170, -2.5170, -2.1990, -1.5619, -2.5170, -1.6340, -2.5170,
         -2.5170],
        [-2.5170, -2.5170, -2.5170, -2.1990, -1.5619, -2.5170, -1.6340, -2.5170,
         -2.5170],
        [-2.5170, -2.5170, -2.5170, -2.1990, -1.5619, -2.5170, -1.6340, -2.5170,
         -2.5170],
        [-2.5170, -2.5170, -2.5170, -2.1990, -1.5619, -2.5170, -1.6340, -2.5170,
         -2.5170],
        [-2.5170, -2.5170, -2.5170, -2.1990, -1.5619, -2.5170, -1.6340, -2.5170,
         -2.5170],
        [-2.5170, -2.5170, -2.5170, -2.1990, -1.5619, -2.5170, -1.6340, -2.5170,
         -2.5170]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[2.2368e-03, 4.4927e-02, 8.2286e-04, 2.2368e-03, 4.4927e-02, 8.2286e-04,
         8.2286e-04, 9.0238e-01, 8.2286e-04],
        [1.0373e-03, 2.81

tensor([[-2.6867, -2.6867, -2.6867, -2.1580, -1.3499, -2.6867, -1.5300, -2.6867,
         -2.6867],
        [-2.6867, -2.6867, -2.6867, -2.1580, -1.3499, -2.6867, -1.5300, -2.6867,
         -2.6867],
        [-2.6867, -2.6867, -2.6867, -2.1580, -1.3499, -2.6867, -1.5300, -2.6867,
         -2.6867],
        [-2.6867, -2.6867, -2.6867, -2.1580, -1.3499, -2.6867, -1.5300, -2.6867,
         -2.6867],
        [-2.6867, -2.6867, -2.6867, -2.1580, -1.3499, -2.6867, -1.5300, -2.6867,
         -2.6867],
        [-2.6867, -2.6867, -2.6867, -2.1580, -1.3499, -2.6867, -1.5300, -2.6867,
         -2.6867],
        [-2.6867, -2.6867, -2.6867, -2.1580, -1.3499, -2.6867, -1.5300, -2.6867,
         -2.6867],
        [-2.6867, -2.6867, -2.6867, -2.1580, -1.3499, -2.6867, -1.5300, -2.6867,
         -2.6867]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[2.0434e-08, 3.3258e-03, 4.5010e-04, 4.9359e-01, 9.0404e-03, 2.7655e-09,
         2.7655e-09, 5.5546e-08, 4.9359e-01],
        [8.2172e-08, 2.44

tensor([[-2.6849, -2.6849, -2.6849, -2.2028, -1.4217, -2.6849, -1.4319, -2.6849,
         -2.6849],
        [-2.6849, -2.6849, -2.6849, -2.2028, -1.4217, -2.6849, -1.4319, -2.6849,
         -2.6849],
        [-2.6849, -2.6849, -2.6849, -2.2028, -1.4217, -2.6849, -1.4319, -2.6849,
         -2.6849],
        [-2.6849, -2.6849, -2.6849, -2.2028, -1.4217, -2.6849, -1.4319, -2.6849,
         -2.6849],
        [-2.6849, -2.6849, -2.6849, -2.2028, -1.4217, -2.6849, -1.4319, -2.6849,
         -2.6849],
        [-2.6849, -2.6849, -2.6849, -2.2028, -1.4217, -2.6849, -1.4319, -2.6849,
         -2.6849],
        [-2.6849, -2.6849, -2.6849, -2.2028, -1.4217, -2.6849, -1.4319, -2.6849,
         -2.6849],
        [-2.6849, -2.6849, -2.6849, -2.2028, -1.4217, -2.6849, -1.4319, -2.6849,
         -2.6849]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[5.2149e-04, 3.8533e-03, 1.4175e-03, 5.2149e-04, 5.2149e-04, 2.1038e-01,
         5.7188e-01, 2.1038e-01, 5.2149e-04],
        [8.3151e-07, 2.06

...

Train Epoch: 95 [80/320 (25%)]	Loss: 0.171153
Train Epoch: 95 [120/320 (38%)]	Loss: 0.139045
Train Epoch: 95 [160/320 (50%)]	Loss: 0.162898
Train Epoch: 95 [200/320 (62%)]	Loss: 0.148136
Train Epoch: 95 [240/320 (75%)]	Loss: 0.163080
Train Epoch: 95 [280/320 (88%)]	Loss: 0.150717
tensor([[-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -

Train Epoch: 96 [80/320 (25%)]	Loss: 0.110377
Train Epoch: 96 [120/320 (38%)]	Loss: 0.145519
Train Epoch: 96 [160/320 (50%)]	Loss: 0.189464
Train Epoch: 96 [200/320 (62%)]	Loss: 0.154396
Train Epoch: 96 [240/320 (75%)]	Loss: 0.153046
Train Epoch: 96 [280/320 (88%)]	Loss: 0.135921
tensor([[-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -

Train Epoch: 97 [80/320 (25%)]	Loss: 0.141891
Train Epoch: 97 [120/320 (38%)]	Loss: 0.176366
Train Epoch: 97 [160/320 (50%)]	Loss: 0.110823
Train Epoch: 97 [200/320 (62%)]	Loss: 0.175118
Train Epoch: 97 [240/320 (75%)]	Loss: 0.145679
Train Epoch: 97 [280/320 (88%)]	Loss: 0.150915
tensor([[-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -

Train Epoch: 98 [80/320 (25%)]	Loss: 0.174444
Train Epoch: 98 [120/320 (38%)]	Loss: 0.167175
Train Epoch: 98 [160/320 (50%)]	Loss: 0.165098
Train Epoch: 98 [200/320 (62%)]	Loss: 0.161451
Train Epoch: 98 [240/320 (75%)]	Loss: 0.161799
Train Epoch: 98 [280/320 (88%)]	Loss: 0.174831
tensor([[-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -

Train Epoch: 99 [80/320 (25%)]	Loss: 0.155816
Train Epoch: 99 [120/320 (38%)]	Loss: 0.166395
Train Epoch: 99 [160/320 (50%)]	Loss: 0.136422
Train Epoch: 99 [200/320 (62%)]	Loss: 0.130735
Train Epoch: 99 [240/320 (75%)]	Loss: 0.160149
Train Epoch: 99 [280/320 (88%)]	Loss: 0.136051
tensor([[-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -2.9805, -2.9805, -2.1313, -1.3816, -2.5100, -1.4891, -2.1183,
         -2.9805],
        [-2.9805, -

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
torch.save(model.state_dict(), 'dataset_model_soundemotion.pt')

## NOTES

below is the mfccs notes / random code

In [13]:
audio, sample_rate = librosa.load("SongEmotionDataset/1.mp3", res_type='kaiser_fast')
# [print(x) for x in audio]

#convert audio into 2d array
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
# mfccsscaled = np.mean(mfccs.T,axis=0)
print(mfccs.shape, audio.shape)
mfccs

/home/czf/memories/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(40, 2586) (1323648,)


array([[-5.30341797e+02, -4.07741577e+02, -3.27536621e+02, ...,
        -2.39811523e+02, -1.96744080e+02, -1.44711777e+02],
       [ 5.81265569e-01,  1.03006027e+02,  1.29354553e+02, ...,
         1.48707626e+02,  1.45873001e+02,  1.28202530e+02],
       [ 4.58764762e-01,  7.53921986e+00, -1.18814125e+01, ...,
        -2.51551704e+01, -1.92207527e+01, -1.79366188e+01],
       ...,
       [ 3.11299562e-01, -1.29907084e+00,  1.18818974e+00, ...,
        -6.58579540e+00, -3.34302998e+00, -4.75482178e+00],
       [ 2.23848164e-01, -3.19489312e+00, -2.78556681e+00, ...,
        -1.36089420e+01, -6.40699673e+00, -5.27228928e+00],
       [ 8.67742151e-02,  1.31472754e+00, -1.41885233e+00, ...,
         3.34440261e-01,  1.14392626e+00, -3.62402201e-02]], dtype=float32)

In [10]:
# audio_tensor = torch.tensor(audio)
# audio_tensor
# audio_tensor.shape

In [16]:
# for sound_file in data_path.iterdir():
#     if ".mp3" in str(sound_file):
#         print(sound_file)
#         audio, sample_rate = librosa.load(str(sound_file), res_type='kaiser_fast')
        
    